In [48]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import sys
sys.path.append('../../')

import os
import pandas as pd
import numpy as np
import config
import utils
import torch
import pickle
import train_impf_MLP

from Model import Impf_DNAMLP
from torch.nn.functional import softmax
from utils import impf_make_ndarray_from_csv, get_int_label
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from Dataset import CNS

In [50]:
print(utils.low_perf_groups)

['Glio-neuronal', 'Other glioma']


In [51]:
inner_fold = '1.1'
outer_fold = f"{inner_fold.split('.')[0]}.0"
group = 'Sella'
alg = 'RF'
print(outer_fold)

1.0


In [52]:
dat_cfg = config.data_config
clf_cfg = config.classifier_config
impf_cfg = config.impf_config
TRAIN_CSV_DIR = dat_cfg['TRAIN_CSV_DIR']
TEST_CSV_DIR = dat_cfg['TEST_CSV_DIR']

INNER_FOLD_CSV = os.path.join(TEST_CSV_DIR, group, f'{inner_fold}_test.csv')
OUTER_FOLD_CSV = os.path.join(TEST_CSV_DIR, group, f'{outer_fold}_test.csv')
device = impf_cfg['device']
device

'cpu'

In [53]:
with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], alg, group, f'{outer_fold}_combined.pkl'), 'rb') as handle:
    impf = pickle.load(handle)
print(len(impf))

2095


In [54]:
impf

{'cg26416971',
 'cg18935349',
 'cg05068987',
 'cg04502985',
 'cg15915835',
 'cg14409958',
 'cg01064265',
 'cg11843238',
 'cg27645259',
 'cg25037165',
 'cg15442907',
 'cg00033213',
 'cg04515001',
 'cg02667677',
 'cg16973527',
 'cg13684379',
 'cg17301223',
 'cg14991769',
 'cg27088725',
 'cg05513806',
 'cg04311722',
 'cg02196834',
 'cg22488797',
 'cg02767613',
 'cg07600533',
 'cg10114725',
 'cg24110050',
 'cg25622154',
 'cg06254440',
 'cg23051392',
 'cg09393254',
 'cg19978181',
 'cg25069807',
 'cg03906434',
 'cg25063151',
 'cg21262776',
 'cg16210842',
 'cg14643892',
 'cg04579493',
 'cg13200838',
 'cg00914726',
 'cg03889013',
 'cg18712494',
 'cg19229716',
 'cg17958291',
 'cg13913990',
 'cg07412315',
 'cg14179581',
 'cg01561758',
 'cg23874561',
 'cg05257898',
 'cg09275704',
 'cg13502741',
 'cg19457477',
 'cg19400179',
 'cg26367197',
 'cg03877767',
 'cg06288821',
 'cg27080287',
 'cg17518215',
 'cg09238332',
 'cg05036656',
 'cg11078084',
 'cg08394412',
 'cg05329960',
 'cg01561259',
 'cg085576

In [55]:
df_inner = pd.read_csv(INNER_FOLD_CSV, index_col=0)
df_outer = pd.read_csv(OUTER_FOLD_CSV, index_col=0)
df_inner.shape, df_outer.shape

((53, 10002), (64, 10002))

In [56]:
pd.Series(pd.read_csv(OUTER_FOLD_CSV, index_col=0).iloc[:, -1]).value_counts()

Sella      37
Control    27
Name: class_group, dtype: int64

## Load random test sample and predict

In [57]:
rand = np.random.randint(df_outer.shape[0])
sample = df_outer.iloc[rand, :]
rand, sample

(41,
 cg22054918          0.410679
 cg16476975           0.27912
 cg03940848          0.008961
 cg25570913          0.117987
 cg02809746          0.094717
                    ...      
 cg21100077           0.33479
 cg00703120           0.83155
 cg00540828               1.0
 label          CONTR, INFLAM
 class_group          Control
 Name: 9611518110_R03C01, Length: 10002, dtype: object)

In [58]:
sample = df_outer.loc[:, list(impf)].iloc[rand,:]
label = get_int_label(df_outer.iloc[rand, -1], group)
sample, sample.shape, label

(cg26416971    0.305999
 cg18935349    0.593202
 cg05068987    0.087167
 cg04502985    0.058354
 cg15915835    0.906159
                 ...   
 cg09035925    0.393387
 cg23037321    0.048154
 cg19876814    0.268613
 cg01072076    0.245850
 cg13169484    0.969269
 Name: 9611518110_R03C01, Length: 2095, dtype: float64,
 (2095,),
 1)

In [59]:
sample_features = np.expand_dims(np.array(sample), axis = 0)
sample_features
# sample_features, type(sample_features), sample_features.shape, type(sample_features[0][0])

array([[0.30599856, 0.59320229, 0.08716713, ..., 0.26861303, 0.24584956,
        0.96926854]])

In [60]:
IMPF_MLP_STATE_FILE = os.path.join(impf_cfg['MLP_BEST_STATES_DIR'], alg, group, f'{inner_fold}.pth')

n_classes = impf_cfg['n_classes']
in_features = len(impf)

model = Impf_DNAMLP(in_features, n_classes)
if torch.cuda.is_available():
    model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE))
else:
    model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE, map_location='cpu'))
model.to(device)

Impf_DNAMLP(
  (densenet): Sequential(
    (0): Linear(in_features=2095, out_features=2, bias=True)
  )
)

In [61]:
sample_features.shape

(1, 2095)

In [62]:
probs = softmax(model((torch.Tensor(sample_features).to(device))), dim = 1)
probs

tensor([[0.2612, 0.7388]], grad_fn=<SoftmaxBackward0>)

In [63]:
# epoch = 1
# test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')
# test_labels_int = np.array([get_int_label(label, group) for label in test_labels])
# test_dataset = CNS(test_features, test_labels_int, mode = 'val')
# test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
# criterion = CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr = impf_cfg['mlp_lr'], weight_decay = impf_cfg['mlp_weight_decay'])
# res = train_impf_MLP.impf_val_epoch(epoch, model, test_loader, criterion, impf_cfg['device'])
# test_loss, test_acc, test_me, test_bs, test_auc, test_f1, test_precision, test_recall, test_cfs = res

In [64]:
# res

In [65]:
# train_features, train_labels = impf_make_ndarray_from_csv(group, inner_fold, impf, mode = 'train')
# val_features, val_labels = impf_make_ndarray_from_csv(group, inner_fold, impf, mode = 'test')
# test_features, test_labels = impf_make_ndarray_from_csv(group, outer_fold, impf, mode = 'test')

# train_labels_int = np.array([get_int_label(label, group) for label in train_labels])
# val_labels_int = np.array([get_int_label(label, group) for label in val_labels])
# test_labels_int = np.array([get_int_label(label, group) for label in test_labels])

# train_dataset = CNS(train_features, train_labels_int, mode = 'train')
# val_dataset = CNS(val_features, val_labels_int, mode = 'val')
# test_dataset = CNS(test_features, test_labels_int, mode = 'val')
# train_loader = DataLoader(train_dataset, batch_size = impf_cfg['mlp_train_batch_size'], shuffle = True)
# val_loader = DataLoader(val_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)
# test_loader = DataLoader(test_dataset, batch_size = impf_cfg['mlp_val_batch_size'], shuffle = False)

# # Init model object
# in_features = len(impf)
# if group in utils.low_perf_groups:
#     model = Impf_GlioMLP(in_features, impf_cfg['n_classes'])
# else:
#     model = Impf_DNAMLP(in_features, impf_cfg['n_classes'])

In [66]:
# %%capture
# all_res = {}
# for fold in utils.inner_folds:
#     res = train_impf_MLP.impf_run(group, alg, fold, train_loader, val_loader, test_loader, model, criterion, optimizer, impf_cfg, 'no_save')
#     all_res[f'{fold}'] = res

In [67]:
# all_res